## Using Tensorflow like Numpy

### Tensors and Operations

In [ ]:
import tensorflow as tf

In [ ]:
tf.constant([[1, 2, 3], [4, 5, 5]]) # matrix

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 5]])>

In [ ]:
tf.constant(40) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=40>

In [ ]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t.shape, t.dtype

(TensorShape([2, 3]), tf.float32)

In [ ]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [ ]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [ ]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [ ]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [ ]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

#### Keras's Low Level API

In [ ]:
K = tf.keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

### Tensors And NumPy

In [ ]:
import numpy as np
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [ ]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [ ]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [ ]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

In [ ]:
# Type Conversion

# tf.constant(2.) + tf.constant(40) # Gives Error
# tf.constant(2.) + tf.constant(40., dtype=tf.float64) # Also Give Error
tf.constant(2.0) + tf.cast(tf.constant(40.), dtype=tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### Variables

In [ ]:
# Tensors can't be changed so we use something called

v = tf.Variable([[1., 2., 3.], [4. ,5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [ ]:
# Modifying Tensors

v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [ ]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,   4.,   6.],
       [  8.,  10., 200.]], dtype=float32)>

### Other Data Structures
* Sparse tensors(tf.SparseTensor) -> efficiently represent tensors using mostly zeroes.
* Tensor Array(tf.TensorArray) -> lists of tensors. all have same shape and same type. Could Be Dynamic
* Ragged Tensor(tf.RaggedTensor) -> same as Tensor Array but Static
* String Tensor(tf.string) -> they represent byte strings. can be converted into Unicode
* Sets(tf.sets) -> can be represented by regular tensor or sparse tensor
* Queues (tf.queue)

## Customizing Models and Training Algorithms

### Custom Loss Functions

In [ ]:
import tensorflow as tf

def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) * 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)


In [ ]:
import numpy as np
X_train = np.arange(0, 100) * 3 + 2
X_train = X_train.reshape((100, 1)).astype(dtype=np.float32)
y_train = np.arange(0, 100, dtype=np.float32)

In [ ]:
model = tf.keras.Sequential([tf.keras.layers.Dense(10, activation="relu"), 
                             tf.keras.layers.Dense(1)])
model.compile(loss=huber_fn, optimizer="nadam", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, verbose=0)

In [ ]:
model.save("test_model.h5")

### Saving and Loading Models That Contain Custom Components

In [ ]:
model = tf.keras.models.load_model("test_model.h5", 
                                   custom_objects={"huber_fn": huber_fn})

In [ ]:
# custom loss function with some different threshold

def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss=create_huber(2.0), optimizer="nadam")
model.fit(X_train, y_train, epochs=10, verbose=0)
model.save("test_model2.h5")

In [ ]:
# load model of custom threshold loss
model = tf.keras.models.load_model("test_model2.h5", 
                                   custom_objects={"huber_fn": create_huber(2.0)})

In [ ]:
# Creating Loss function with keras.losses.Loss Subclass

class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold*tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold":self.threshold}
    

In [ ]:
model.compile(loss=HuberLoss(2.), optimizer="nadam")
model.fit(X_train, y_train, epochs=10, verbose=0)
model.save("test_model3.h5")

In [ ]:
# model = tf.keras.models.load_model("test_model3.h5", 
#                                    custom_objects={"HuberLoss", HuberLoss})

### Custom Activation Function, Initializer, Regularizer, and Constraints

In [ ]:
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights):
    return tf.where(weights < 0, tf.zeros_like(weights), weights)

In [ ]:
layer = tf.keras.layers.Dense(30, activation=my_softplus, 
                              kernel_initializer=my_glorot_initializer, 
                              kernel_regularizer=my_l1_regularizer, 
                              kernel_constraint=my_positive_weights)

In [ ]:
# custom l1 regualrizer with subclassing

class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(0.01 * weights))
    def get_config(self):
        return {"factor": self.factor}

### Custom  Metrics

In [ ]:
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)])

In [ ]:
precision = tf.keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [ ]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [ ]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [ ]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [ ]:
precision.reset_states()

In [ ]:
# Custom Metrics using SubClassing

class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

### Custom layers

In [2]:
# custom layers with no weights
import tensorflow as tf
from tensorflow.keras import layers

layer = layers.Lambda(lambda x: tf.exp(x))

In [ ]:
# custom dense layer with subclassing
class MyDense(layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(name="kernel", shape=[batch_input_shape[-1], self.units], 
                                      initializer="glorot_normal")
        self.bias = self.add_weight(name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)
    
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units, 
                "activation": tf.keras.activations.get(self.activation)}
    
        

In [ ]:
# custom layer with multiple inputs

class MyMultiLayer(layers.Layer):
    def call(self, X):
        X1, X2 = X
        return [X1 + X2, X1 * X2, X1 / X2]
    
    def compute_output_shape(self, batch_input_shape):
        b1, b2 = batch_input_shape
        return [b1, b1, b1]


In [ ]:
# custom layer with different behaivour while training
class MyGaussianNoise(layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
    
    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        return X
    
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape
    

### Custom Models

In [3]:
class ResidualBlock(layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [layers.Dense(n_neurons, activation="elu", 
                                    kernel_initializer="he_normal") 
        for _ in range(n_layers)]
    
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z
    

In [ ]:
# custom model
class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = layers.Dense(30, activation="elu", 
                                   kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = layers.Dense(output_dim)
    
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)
    

### Losses and Metrics Based on Model internals

In [ ]:
class ReconstructionRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [layers.Dense(30, activation="selu", 
                                    kernel_initializer="lecun_normal") for _ in range(5)]
        self.out = layers.Dense(output_dim)
    
    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[:-1]
        self.reconstruct = layers.Dense(n_inputs)
        super().build(batch_input_shape)
        
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        return self.out(Z)


In [ ]:
import numpy as np
X = np.expand_dims(3 * np.arange(0, 100) + np.random.randn(100), axis=1)
y = np.arange(0, 100)

### Custom Training Loops

In [ ]:
l2_reg = tf.keras.regularizers.l2(0.05)
model = tf.keras.Sequential([
    layers.Dense(30, activation="elu", kernel_initializer="he_normal", 
                 kernel_regularizer=l2_reg), 
    layers.Dense(1, kernel_regularizer=l2_reg)
])

In [ ]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [ ]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end=end)

In [ ]:
import numpy as np

X_train = np.expand_dims(3 * np.arange(0, 1000) + 4, axis=1)
y_train = np.arange(0, 1000)

In [ ]:
X_train[:10], y_train[:10]

(array([[ 4],
        [ 7],
        [10],
        [13],
        [16],
        [19],
        [22],
        [25],
        [28],
        [31]]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

In [ ]:
# hyperparameters
n_epochs = 20
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = tf.keras.losses.mean_squared_error
mean_loss = tf.keras.metrics.Mean()
metrics = [tf.keras.metrics.MeanAbsoluteError()]

In [ ]:
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        # Used When Using Kernel_constriants in layers while creating layer
        # for variable in model.variables:
        #   if variable.constraint is not None:
        #       variable.assign(variable.constraint(variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

Epoch 1/20
1000/1000 - mean: 652224.5625 - mean_absolute_error: 390.40272
Epoch 2/20
1000/1000 - mean: 146244.1719 - mean_absolute_error: 123.4486
Epoch 3/20
1000/1000 - mean: 143371.0156 - mean_absolute_error: 120.3137
Epoch 4/20
1000/1000 - mean: 145454.6250 - mean_absolute_error: 124.0024
Epoch 5/20
1000/1000 - mean: 145320.7812 - mean_absolute_error: 123.1653
Epoch 6/20
1000/1000 - mean: 140095.8125 - mean_absolute_error: 120.6189
Epoch 7/20
1000/1000 - mean: 140602.5625 - mean_absolute_error: 119.2709
Epoch 8/20
1000/1000 - mean: 143449.7656 - mean_absolute_error: 120.5120
Epoch 9/20
1000/1000 - mean: 141065.4688 - mean_absolute_error: 118.8797
Epoch 10/20
1000/1000 - mean: 135523.9688 - mean_absolute_error: 113.4624
Epoch 11/20
1000/1000 - mean: 135005.6094 - mean_absolute_error: 113.3805
Epoch 12/20
1000/1000 - mean: 142380.9688 - mean_absolute_error: 117.3519
Epoch 13/20
1000/1000 - mean: 134118.3281 - mean_absolute_error: 113.9107
Epoch 14/20
1000/1000 - mean: 137253.5156 - me

## Tensorflow Functions and Graphs

In [ ]:
cube = lambda x: x**3
cube(2)

8

In [ ]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [ ]:
# python function to tensorflow function
tf_cube = tf.function(cube)
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [ ]:
# we can also use tensorflow decorator for this
@tf.function
def tf_cube(x): return x**3

In [ ]:
# getting orginal python function from tensorflow function
tf_cube.python_function(2)

8

In [ ]:
tf.autograph.to_code(cube)

"tf__lam = (lambda x: ag__.with_function_scope((lambda lscope: (x ** 3)), 'lscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)))\n"

## Exercise

### Custom Layer Normalization

In [ ]:
from tensorflow import keras
import tensorflow as tf

class MyLayerNormalization(keras.layers.Layer):
    def __init__(self, epsilon=0.001, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        
    def build(self, input_shape):
        shape = input_shape[-1:]
        self.alpha = self.add_weight(name="alpha", initializer="ones", 
                                     shape=shape, dtype=tf.float32)
        self.beta = self.add_weight(name="beta", initializer="zeros", 
                                    shape=shape, dtype=tf.float32)
        super().build(input_shape)
        
    def call(self, X, y=None, training=None):
        mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        sd = tf.math.sqrt(variance)
        return self.alpha * (X - mean)/(sd + self.epsilon) + self.beta
    
    def conpute_output_shape(self, input_shape):
        return input_shape
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "epsilon": self.epsilon}


In [ ]:

from sklearn.datasets import make_regression
import numpy as np

X, y = make_regression(n_samples=1000, n_features=5)
X = X.astype(np.float32)

In [ ]:
X[:5], y[:5]

(array([[-5.1069832e-01, -5.7739085e-01, -9.4291204e-01, -4.5033672e-01,
         -4.8785728e-01],
        [ 9.8259677e-04, -7.8620315e-01,  8.7763184e-01, -4.4420934e-01,
         -6.3795716e-01],
        [-5.8136702e-01,  3.4705624e-01,  6.8357337e-01,  9.5518053e-01,
         -1.7072307e+00],
        [ 6.1355406e-01,  1.4041770e+00,  1.8225477e+00,  2.2782083e-01,
         -4.1533902e-01],
        [-5.3237361e-01, -9.5471901e-01, -6.1224246e-01, -8.7758839e-01,
         -5.8460987e-01]], dtype=float32),
 array([-134.82821596,  -26.33945562,   44.05531636,  137.85072483,
        -161.68674774]))

In [ ]:
custom_norm_layer = MyLayerNormalization()
keras_norm_layer = keras.layers.LayerNormalization()
tf.reduce_mean(keras.losses.mean_absolute_error(custom_norm_layer(X), keras_norm_layer(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=0.00046119821>

In [ ]:
model = keras.Sequential([
    layers.Dense(10), 
    MyLayerNormalization(), 
    layers.Dense(1)
])
model.compile(loss="mae", optimizer="sgd")
model.fit(X, y, epochs=10)

Epoch 1/10
32/32 [==============================] - 0s 1ms/step - loss: 97.4643
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 95.6691
Epoch 3/10
32/32 [==============================] - 0s 1ms/step - loss: 92.0266
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 79.2323
Epoch 5/10
32/32 [==============================] - 0s 2ms/step - loss: 53.8663
Epoch 6/10
32/32 [==============================] - 0s 2ms/step - loss: 35.3591
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 26.0753
Epoch 8/10
32/32 [==============================] - 0s 2ms/step - loss: 22.0763
Epoch 9/10
32/32 [==============================] - 0s 2ms/step - loss: 19.4397
Epoch 10/10
32/32 [==============================] - 0s 2ms/step - loss: 17.2413


In [ ]:
model = keras.Sequential([
    layers.Dense(10), 
    layers.LayerNormalization(), 
    layers.Dense(1)
])
model.compile(loss="mae", optimizer="sgd")
model.fit(X, y, epochs=10)

Epoch 1/10
32/32 [==============================] - 0s 731us/step - loss: 96.8731
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 94.8426
Epoch 3/10
32/32 [==============================] - 0s 1ms/step - loss: 89.8132
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 73.1531
Epoch 5/10
32/32 [==============================] - 0s 913us/step - loss: 50.2599
Epoch 6/10
32/32 [==============================] - 0s 1ms/step - loss: 33.2309
Epoch 7/10
32/32 [==============================] - 0s 1ms/step - loss: 25.4041
Epoch 8/10
32/32 [==============================] - 0s 1ms/step - loss: 21.8266
Epoch 9/10
32/32 [==============================] - 0s 2ms/step - loss: 18.9184
Epoch 10/10
32/32 [==============================] - 0s 1ms/step - loss: 16.5698


### Train Model Using Custom Training loop

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_val, y_train, y_val = X_train[:50000], X_train[50000:], y_train[:50000], y_train[50000:]

In [ ]:
from tensorflow.keras import layers, optimizers, Sequential, losses, metrics
import numpy as np

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model = Sequential([
    layers.Flatten(input_shape=(28, 28)), 
    layers.Dense(300, activation="elu", kernel_initializer="he_normal"), 
    layers.BatchNormalization(), 
    layers.Dense(100, activation="elu", kernel_initializer="he_normal"), 
    layers.BatchNormalization(), 
    layers.Dense(100, activation="elu", kernel_initializer="he_normal"), 
    layers.BatchNormalization(), 
    layers.Dense(10, activation="softmax")
], name="fashion_mnist_model")

In [ ]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

In [ ]:
def print_status_bar(iteration, total, loss, metrics=None, size=30):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(iteration, total) + metrics, end=end)

In [ ]:
n_epochs = 100
batch_size = 32
n_steps = len(X) // batch_size
optimizer = optimizers.Adam(learning_rate=1e-3)
loss_fn = losses.sparse_categorical_crossentropy
mean_loss = metrics.Mean()
metrics = [metrics.Accuracy()]

In [ ]:
for epoch in range(1, n_epochs+1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps+1):
        X_batch, y_batch = random_batch(X_train, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, tf.argmax(y_pred, axis=1))
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

Epoch 1/100
50000 - 50000mean: 39.6706 - accuracy: 0.4899
Epoch 2/100
50000 - 50000mean: 15.9651 - accuracy: 0.5978
Epoch 3/100
50000 - 50000mean: 10.7381 - accuracy: 0.6714
Epoch 4/100
50000 - 50000mean: 6.3557 - accuracy: 0.6986
Epoch 5/100
50000 - 50000mean: 4.8818 - accuracy: 0.6956
Epoch 6/100
50000 - 50000mean: 4.0036 - accuracy: 0.7006
Epoch 7/100
50000 - 50000mean: 3.4636 - accuracy: 0.7288
Epoch 8/100
50000 - 50000mean: 3.2233 - accuracy: 0.7157
Epoch 9/100
50000 - 50000mean: 3.0913 - accuracy: 0.6976
Epoch 10/100
50000 - 50000mean: 3.6358 - accuracy: 0.6593
Epoch 11/100
50000 - 50000mean: 2.0224 - accuracy: 0.7097
Epoch 12/100
50000 - 50000mean: 1.8251 - accuracy: 0.7026
Epoch 13/100
50000 - 50000mean: 2.0329 - accuracy: 0.7137
Epoch 14/100
50000 - 50000mean: 1.3845 - accuracy: 0.7228
Epoch 15/100
50000 - 50000mean: 1.5505 - accuracy: 0.6976
Epoch 16/100
50000 - 50000mean: 1.2465 - accuracy: 0.7157
Epoch 17/100
50000 - 50000mean: 1.3590 - accuracy: 0.7097
Epoch 18/100
50000 -

In [ ]:
tf.argmax(y_pred, axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 8, 5, 5, 5], dtype=int64)>

In [ ]:
model_2 = model = Sequential([
    layers.Flatten(input_shape=(28, 28)), 
    layers.Dense(300, activation="elu", kernel_initializer="he_normal"), 
    layers.BatchNormalization(), 
    layers.Dense(100, activation="elu", kernel_initializer="he_normal"), 
    layers.BatchNormalization(), 
    layers.Dense(100, activation="elu", kernel_initializer="he_normal"), 
    layers.BatchNormalization(), 
    layers.Dense(10, activation="softmax")
], name="fashion_mnist_model_2")
model_2.compile(loss="sparse_categorical_crossentropy", 
                optimizer="adam", 
                metrics=["accuracy"])
model_2.fit(X_train, y_train, epochs=100, 
            validation_data=(X_val, y_val), 
            callbacks=[keras.callbacks.EarlyStopping(patience=10, 
                                                     monitor="val_loss",
                                                     restore_best_weights=True)])

Epoch 1/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.4897 - accuracy: 0.8247 - val_loss: 0.4302 - val_accuracy: 0.8430
Epoch 2/100
1563/1563 [==============================] - 4s 3ms/step - loss: 0.3931 - accuracy: 0.8559 - val_loss: 0.3816 - val_accuracy: 0.8683
Epoch 3/100
1563/1563 [==============================] - 4s 3ms/step - loss: 0.3646 - accuracy: 0.8650 - val_loss: 0.4724 - val_accuracy: 0.8619
Epoch 4/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.3380 - accuracy: 0.8752 - val_loss: 0.4959 - val_accuracy: 0.8720
Epoch 5/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.3184 - accuracy: 0.8818 - val_loss: 0.6493 - val_accuracy: 0.8757
Epoch 6/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.3016 - accuracy: 0.8885 - val_loss: 0.5066 - val_accuracy: 0.8644
Epoch 7/100
1563/1563 [==============================] - 4s 2ms/step - loss: 0.2863 - accuracy: 0.8920 - val_loss: 0.4828 - val_ac